In [1]:
from data_manipulation import fm_create_dataframe, concat_positions
import sqlite3
import pandas as pd
from typing import List, Union, Any, Tuple

df = fm_create_dataframe(path='data/SERIEC_2024.html')
df = df.drop(columns=['Melhor Pos','Posição Sec.'])

In [ ]:
colunas = df.columns.to_list()
colunas

In [ ]:
list(df.itertuples(index=False, name=None))

In [2]:


def connect_db(name: str = 'data.db') -> sqlite3.Connection:
   
    path_db = f'/home/mjsa/Github/fm_analise_de_dados/data/db/{name}' #Ajustar quando adicionado ao programa
    
    conn = sqlite3.connect(path_db)
    
    return conn

In [3]:
def create_db(table_name: str = 'stats') -> None:

    # path_db = f'/home/mjsa/Github/fm_analise_de_dados/data/db/data.db'
    
    # conn = sqlite3.connect(path_db)
    conn = connect_db()
    cur = conn.cursor()
    
    scheme = f""" 
    
    CREATE TABLE IF NOT EXISTS {table_name} (
    
    id INTEGER,
    nome TEXT,
    nac TEXT,
    idade INTEGER,
    altura TEXT,
    posicao TEXT,
    melhor_pe TEXT,
    person TEXT,
    clube TEXT,
    divisao TEXT,
    salario INTEGER,
    final_contrato TEXT,
    partidas TEXT,
    minutos INTEGER,
    nota_med REAL,
    motm REAL,
    ass REAL,
    gols REAL,
    gk_sg REAL,
    gk_gsof_p90 REAL,
    xG REAL,
    npxG REAL,
    chutes_p90 REAL,
    passe_t_p90 REAL,
    passe_c_p90 REAL,
    chutes_gol_p90 REAL,
    press_t_p90 REAL,
    press_c_p90 REAL,
    poss_g_p90 REAL,
    poss_p_p90 REAL,
    des_c_p100 REAL,
    passe_dec_p90 REAL,
    jg_ar_t_p90 REAL,
    cab_g_p100 REAL,
    int_p90 REAL,
    alivios_p90 REAL,
    ass_p90 REAL,
    bloqueios_p90 REAL,
    cab_dec_p90 REAL,
    cab_g_p90 REAL,
    cab_p_p90 REAL,
    cruz_c_p90 REAL,
    cruz_t_p90 REAL,
    des_dec_p90 REAL,
    fintas_p90 REAL,
    passe_prog_p90 REAL,
    passe_c_p100 REAL,
    xA_p90 REAL,
    npxG_p90 REAL,
    grandes_chances REAL,
    cruz_c_p100 REAL,
    gk_xG_def_p90 REAL,
    gk_xG_def REAL,
    xA REAL,
    gk_def_p90 REAL,
    gk_pen_def_p100 REAL,
    des_g_p90 REAL,
    id_temporada REAL,
    preco_min REAL,
    preco_max REAL,
    coef REAL,
    posicao_analise TEXT,
    salario_anual REAL,
    grandes_chances_p90 REAL,
    aval_cria REAL,
    np_chutes REAL,
    np_chutes_p90 REAL,
    np_chutes_gol REAL,
    np_chutes_gol_p90 REAL,
    np_chutes_gol_p100 REAL,
    xG_p90 REAL,
    npG REAL,
    npG_p90 REAL,
    conv_p100 REAL,
    npG_ae REAL,
    conv_penal_p100 REAL,
    npxG_per_np_chute REAL,
    xPnpG_p90 REAL,
    pnpG_p90 REAL,
    aval_fin REAL,
    aof_p90 REAL,
    faltas_sofridas_p90 REAL,
    erros_decisivos_p90 REAL,
    gk_def_dif_p90 REAL,
    gk_def_segu_p90 REAL,
    gk_def_desv_p90 REAL,
    des_t_p90 REAL,
    duel_t_p90 REAL,
    duel_g_p90 REAL,
    rtg_duel REAL,
    adef_t_p90 REAL,
    adef_c_p90 REAL,
    rtg_adef REAL,
    rtg_jg_ar REAL,
    rtg_des REAL,
    rtg_rec_bola REAL,
    aval_def REAL,
    source TEXT,
    PRIMARY KEY (id, id_temporada)
    );
    """
    
    try:
        cur.execute(scheme)
        conn.commit()
        
    except Exception as e:
        print(f'{e}')
        
    finally:
        conn.close()
    

In [5]:

def bulk_upsert(df: pd.DataFrame, db_name: str = 'data.db', table_name: str = 'stats') -> None:
    
    columns_df = list(df.columns)
    conn = connect_db(name = db_name)
    cursor = conn.cursor()
    
    if "posicao_analise" in columns_df:
        df['posicao_analise'] = df['posicao_analise'].apply(concat_positions)
    else:
        return
    
    
    placeholders = ", ".join(["?"]*len(columns_df))
    columns_joined = ", ".join(columns_df)
    update_columns = [f"{col}=excluded.{col}" for col in columns_df if col not in ("id","id_temporada")]
    update_joined = ", ".join(update_columns)
    
    query = f"""
    INSERT INTO {table_name} ({columns_joined})
    VALUES ({placeholders})
    ON CONFLICT(id, id_temporada) DO UPDATE SET
    {update_joined};
    """

    values = list(df.itertuples(index=False, name=None))
    
    try:
        
        cursor.executemany(query, values)
        conn.commit()
        
    except Exception as e:
        print({e})
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()   

In [6]:
def clear_table(table_name: str = 'stats')-> None:
    conn = connect_db()
    cursor = conn.cursor()
    
    try:
        
        cursor.execute(f"DELETE FROM {table_name}")
        conn.commit()
    
    except Exception as e:
        print(f"{e}")
        
        if conn:
            conn.rollback()
    
    finally:
        if conn:
            conn.close()
            

In [7]:
def delete_rows(column_name: str, list_values: List[Any],table_name: str = 'stats') -> None:
    
    conn = connect_db()
    cursor = conn.cursor()
    
    try:
        
        placeholders = ", ".join(["?"]*len(list_values))
        
        query = f"""
        DELETE FROM {table_name} 
        WHERE  {column_name} in ({placeholders})
        """
        cursor.execute(query, list_values)
        conn.commit()
    
    except Exception as e:
        print(f"{e}")
        
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

In [25]:
create_db()

In [8]:
bulk_upsert(df)

In [8]:
delete_rows('source',['data/SERIEC_2024.html'])